In [1]:
%load_ext autoreload
%autoreload 2

from llama3.eval import Experiment
from llama3.utils.load_utils import load_config, save_config, setup_log_dir

config = load_config("configs/eval_bayes_ensemble.yaml")

experiment_log_dir = setup_log_dir(
    config.get("logs_dir", "experiment_logs"), experiment_name=config.get("experiment_name", None)
)

config["experiment_config"]["experiment_log_dir"] = experiment_log_dir
save_config(config.to_dict(), experiment_log_dir + "/config.yaml")

experiment = Experiment(config["experiment_config"])

/home/paperspace/miniconda/envs/posteriors/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/paperspace/miniconda/envs/posteriors/lib/python3.11/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.11s/it]
Some weights of BayesLlamaForCausalLM were not initialized from the model checkpoint at Meta-Llama-3-8B-Instruct and are newly initialized: ['model.bayesian_layers.0.input_layernorm.weight', 'model.bayesian_layers.0.mlp.down_proj.weight', 'model.bayesian_layers.0.mlp.gate_proj.weight', 'model.bayesian_layers.0.mlp.up_proj.weight', 'model.bayesian_layers.

In [4]:
dset_name = "head_qa"
experiment.run(dset_name, config["dataset_path"][dset_name][0])

  0%|          | 0/12 [00:00<?, ?it/s]

[' 2. Colágeno.'] ['2. Colágeno.']
[' 1. Mitocondrial interna.'] ['1. Mitocondrial interna.']
[' 4. Locomoci$MESS'] ['4. Locomoción celular.']
[' 4. Endos\ufeff\n\n'] ['4. Endosomas.']
[' 2. Celsula de Schwann.'] ['2. Células de Schwann.']
[' 3. Discos intervertebris. Click'] ['3. Discos intervertebrales.']
[' 1. Nucleosoma.'] ['3. Cromosoma eucariótico.']
[' 1. Condrobl'] ['4. Osteoclastos.']
[' 1. Conjuntiva laxa. The'] ['4. Cartílago.']
[' 3. Endocondral.'] ['3. Endocondral.']
[' 1. Proteosomas. '] ['1. Proteosomas.']
[' 2. Membrana plHeaderCode'] ['1. Retículo endoplásmico.']
[' 3. Arp 2/3.'] ['3. Arp 2/3.']
[' 1. Axonema.'] ['2. Centrosoma.']
[' 2. CromátGuidId'] ['4. Cromosomas homólogos.']
[' 3. Ácid hialurónico.'] ['3. Ácido hialurónico.']
[' 1. Neutropilio.'] ['1. Neutrófilo.']
[' 2. Conjuntivo.'] ['2. Conjuntivo.']
[' 4. Liso visceral. Click image to'] ['2. Cardiaco.']
[' 1. Cerebro.'] ['4. Médula espinal.']
[' 4. Q'] ['4. Una de las funciones de las proteínas transmembrana e

  0%|          | 0/12 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [11]:
import torch

inputs_a = """
Answer the following multiple choice question as succintly as possible.

Steps of the scientific method include all of the following except
a. doing background research.
b. constructing a hypothesis.
c. asking a question.
d. proving a theory.

Answer:
"""
inputs_b = """
Answer the following multiple choice question as succintly as possible.

Why do scientists call the Big Bang a theory?
a. It is probably unlikely and therefore not a fact.
b. A very well respected scientist proved it to be true.
c. Many scientists have agreed upon this explanation after repeated experiments and models have shown it
d. All possible answers to a scientific idea are called theories.

Answer:
"""
inputs_c = """
Answer the following multiple choice question as succintly as possible.

The data collected in an experiment should always be
a. labeled.
b. recorded.
c. reported.
d. all of the above

Answer:
"""
inputs_d = """
Answer the following multiple choice question as succintly as possible.

If the results of an experiment disprove a hypothesis, then the
a. results should not be reported.
b. hypothesis is just a theory.
c. data must contain errors.
d. none of the above

Answer:
"""

@torch.no_grad()
def generate(model, inputs, max_length=20, use_cache=True):
    seq_out = []
    for idx in range(max_length):
        outputs = model(**inputs, return_dict=False, use_cache=use_cache)

        if "attention_mask" in inputs:
            del inputs["attention_mask"]

        next_token = outputs[0][0][:, -1].argmax(-1).unsqueeze(-1)
        if use_cache:
            inputs["past_key_values"] = outputs[1]
            inputs["ensemble_past_key_values"] = outputs[2]
            inputs["input_ids"] = next_token
        else:
            inputs["input_ids"] = torch.cat([inputs["input_ids"], next_token], dim=1)
        seq_out.append(next_token)

    return torch.cat(seq_out, -1)

inputs = [inputs_a, inputs_b, inputs_c, inputs_d]
inputs = experiment.tokenizer(inputs, padding=True, return_tensors="pt")
seq_out = generate(experiment.model, inputs.to("cuda"), use_cache=True)
experiment.tokenizer.batch_decode(seq_out, skip_special_tokens=True)

['The correct option isFigure ',
 'The correct option is d. All possible answers to a scientific idea are called theories.  The Big',
 'd',
 'd.  ://://Figure  Figure ']